## Meteorology analysis
### Author : Nicolas Lacroix (https://github.com/NicolasLacroix)

#### Analyzed data will be used to predict the Sud region's consumption

Licence : [Apache License 2.0]

Data provided by [OpenDataSoft] and [Météo France]

Source file link (csv) : https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/download/?format=csv&refine.nom_reg=Provence-Alpes-C%C3%B4te+d%27Azur&timezone=Europe/Berlin&use_labels_for_header=true&csv_separator=%3B

[OpenDataSoft]: https://public.opendatasoft.com/
[Météo France]: https://donneespubliques.meteofrance.fr/

[Apache License 2.0]: https://github.com/NicolasLacroix/data-representation/blob/master/LICENSE

In [97]:
# TODO: add inhabitants number to datasets
#%run MyOtherNotebook.ipynb
import pandas as pd
import numpy as np
from datetime import datetime, date
import matplotlib.pyplot as plt

%matplotlib inline

In [98]:
# TODO: read https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/api/
# downloading data for the SUD region
# data_link = "https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/download/?format=csv&refine.nom_reg=Provence-Alpes-C%C3%B4te+d%27Azur&timezone=Europe/Berlin&use_labels_for_header=true&csv_separator=%3B"
data_link = "../Datasets/meteorologie/donnees-synop-essentielles-omm-paca.csv"
data_meteo = pd.read_csv(data_link, delimiter=';', encoding='utf_8', parse_dates=True)
data_meteo['Date'] = pd.to_datetime(data_meteo['Date'], format='%Y-%m-%dT%H:%M:%S', utc=True)
data_meteo['Date'] = data_meteo['Date'].dt.tz_convert('Europe/Paris')
print(data_meteo.shape)
data_meteo.head()

(115233, 81)


,ID OMM station,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Point de rosée,Humidité,...,Longitude,Latitude,communes (name),communes (code),EPCI (name),EPCI (code),department (name),department (code),region (name),region (code)
0,7650,2015-06-08 14:00:00+02:00,101500.0,-110.0,8.0,250.0,3.8,303.15,287.25,38.0,...,5.216000,43.437667,Marignane,13054,Métropole d'Aix-Marseille-Provence,200054807,Bouches-du-Rhône,13,Provence-Alpes-Côte d'Azur,93
1,7591,2015-06-13 11:00:00+02:00,NaN,-90.0,8.0,50.0,3.4,293.35,282.25,49.0,...,6.502333,44.565667,Embrun,5046,CC Serre-Ponçon,200067742,Hautes-Alpes,5,Provence-Alpes-Côte d'Azur,93
2,7591,2015-06-13 14:00:00+02:00,NaN,-100.0,8.0,200.0,4.0,297.45,284.45,44.0,...,6.502333,44.565667,Embrun,5046,CC Serre-Ponçon,200067742,Hautes-Alpes,5,Provence-Alpes-Côte d'Azur,93
3,7591,2015-06-06 23:00:00+02:00,NaN,10.0,0.0,20.0,3.0,289.45,283.95,70.0,...,6.502333,44.565667,Embrun,5046,CC Serre-Ponçon,200067742,Hautes-Alpes,5,Provence-Alpes-Côte d'Azur,93
4,7591,2015-06-09 17:00:00+02:00,NaN,20.0,3.0,280.0,8.6,294.15,281.05,43.0,...,6.502333,44.565667,Embrun,5046,CC Serre-Ponçon,200067742,Hautes-Alpes,5,Provence-Alpes-Côte d'Azur,93


In [99]:
# removing useless columns
# TODO: add EPCI (or department) when inhabitants number will be added
# removing columns with too much NaN values
acceptablePercentage = 40

def removeNaN(threshold, data):
    numRows, numColumns = data.shape
    nan_stats = data.isna().sum()
    for col_name, value in nan_stats.items():
        # if more than the acceptable percentage of the values are NaN
        if value*100/numRows >= acceptablePercentage:
            data = data.drop(columns=[col_name])
    return data

data_meteo = removeNaN(acceptablePercentage, data_meteo)
data_meteo

,ID OMM station,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Point de rosée,Humidité,...,Longitude,Latitude,communes (name),communes (code),EPCI (name),EPCI (code),department (name),department (code),region (name),region (code)
0,7650,2015-06-08 14:00:00+02:00,101500.0,-110.0,8.0,250.0,3.8,303.15,287.25,38.0,...,5.216000,43.437667,Marignane,13054,Métropole d'Aix-Marseille-Provence,200054807,Bouches-du-Rhône,13,Provence-Alpes-Côte d'Azur,93
1,7591,2015-06-13 11:00:00+02:00,NaN,-90.0,8.0,50.0,3.4,293.35,282.25,49.0,...,6.502333,44.565667,Embrun,5046,CC Serre-Ponçon,200067742,Hautes-Alpes,5,Provence-Alpes-Côte d'Azur,93
2,7591,2015-06-13 14:00:00+02:00,NaN,-100.0,8.0,200.0,4.0,297.45,284.45,44.0,...,6.502333,44.565667,Embrun,5046,CC Serre-Ponçon,200067742,Hautes-Alpes,5,Provence-Alpes-Côte d'Azur,93
3,7591,2015-06-06 23:00:00+02:00,NaN,10.0,0.0,20.0,3.0,289.45,283.95,70.0,...,6.502333,44.565667,Embrun,5046,CC Serre-Ponçon,200067742,Hautes-Alpes,5,Provence-Alpes-Côte d'Azur,93
4,7591,2015-06-09 17:00:00+02:00,NaN,20.0,3.0,280.0,8.6,294.15,281.05,43.0,...,6.502333,44.565667,Embrun,5046,CC Serre-Ponçon,200067742,Hautes-Alpes,5,Provence-Alpes-Côte d'Azur,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115228,7690,2019-12-15 10:00:00+01:00,101370.0,240.0,2.0,320.0,4.1,284.65,276.45,57.0,...,7.209000,43.648833,Nice,6088,Métropole Nice Côte d'Azur,200030195,Alpes-Maritimes,6,Provence-Alpes-Côte d'Azur,93
115229,7650,2019-12-15 13:00:00+01:00,101310.0,-30.0,8.0,130.0,5.5,289.95,283.55,66.0,...,5.216000,43.437667,Marignane,13054,Métropole d'Aix-Marseille-Provence,200054807,Bouches-du-Rhône,13,Provence-Alpes-Côte d'Azur,93
115230,7661,2019-12-24 01:00:00+01:00,101980.0,10.0,2.0,300.0,10.1,283.95,271.85,43.0,...,5.940833,43.079333,Saint-Mandrier-sur-Mer,83153,Métropole Toulon-Provence-Méditerranée,248300543,Var,83,Provence-Alpes-Côte d'Azur,93
115231,7650,2019-12-17 10:00:00+01:00,100790.0,-70.0,8.0,100.0,11.3,290.85,282.45,58.0,...,5.216000,43.437667,Marignane,13054,Métropole d'Aix-Marseille-Provence,200054807,Bouches-du-Rhône,13,Provence-Alpes-Côte d'Azur,93


In [100]:
# removing cities' and other useless caracteristics
# TODO: define criterias for remove selection
toRemove = [
    'Coordonnees', 'Nom', 'Longitude', 'Latitude', 'communes (name)', 
    'communes (code)', 'EPCI (name)', 'EPCI (code)', 'department (name)', 
    'department (code)', 'region (name)', 'region (code)',
    'Temps passé 1.1', 'Temps présent.1', 'Altitude', 
    'ID OMM station', 'Visibilité horizontale',
    'Température'
]
# we delete the Temperature column because we want the values in Celcius (not in Kelvin) 
# corresponding of the Température (°C) column
data_meteo = data_meteo.drop(columns=toRemove, errors='ignore') # ignore errors due to previous deletions
data_meteo

,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Point de rosée,Humidité,Temps présent,Pression station,Rafales sur une période,Periode de mesure de la rafale,Précipitations dans la dernière heure,Précipitations dans les 3 dernières heures,Précipitations dans les 6 dernières heures,Précipitations dans les 12 dernières heures,Type de tendance barométrique.1,Température (°C)
0,2015-06-08 14:00:00+02:00,101500.0,-110.0,8.0,250.0,3.8,287.25,38.0,NaN,101140.0,NaN,NaN,0.0,0.0,0.0,0.0,8.0,30.0
1,2015-06-13 11:00:00+02:00,NaN,-90.0,8.0,50.0,3.4,282.25,49.0,0.0,91480.0,7.9,-10.0,0.0,0.0,0.2,9.1,8.0,20.2
2,2015-06-13 14:00:00+02:00,NaN,-100.0,8.0,200.0,4.0,284.45,44.0,0.0,91380.0,7.2,-10.0,0.0,0.0,0.0,0.2,8.0,24.3
3,2015-06-06 23:00:00+02:00,NaN,10.0,0.0,20.0,3.0,283.95,70.0,0.0,92480.0,4.8,-10.0,0.0,0.0,-0.1,-0.1,0.0,16.3
4,2015-06-09 17:00:00+02:00,NaN,20.0,3.0,280.0,8.6,281.05,43.0,0.0,91530.0,16.1,-10.0,0.0,0.0,0.0,0.0,3.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115228,2019-12-15 10:00:00+01:00,101370.0,240.0,2.0,320.0,4.1,276.45,57.0,2.0,101060.0,6.7,-10.0,0.0,0.0,0.0,0.0,2.0,11.5
115229,2019-12-15 13:00:00+01:00,101310.0,-30.0,8.0,130.0,5.5,283.55,66.0,0.0,100930.0,8.0,-10.0,0.0,0.0,0.0,0.0,8.0,16.8
115230,2019-12-24 01:00:00+01:00,101980.0,10.0,2.0,300.0,10.1,271.85,43.0,NaN,100310.0,15.2,-10.0,0.0,0.0,0.0,0.0,2.0,10.8
115231,2019-12-17 10:00:00+01:00,100790.0,-70.0,8.0,100.0,11.3,282.45,58.0,0.0,100410.0,20.5,-10.0,0.0,0.0,0.0,0.0,8.0,17.7


In [101]:
# sort data by date (increasing order)
data_meteo = data_meteo.sort_values(by='Date')
data_meteo.dtypes

Date                                           datetime64[ns, Europe/Paris]
Pression au niveau mer                                              float64
Variation de pression en 3 heures                                   float64
Type de tendance barométrique                                       float64
Direction du vent moyen 10 mn                                       float64
Vitesse du vent moyen 10 mn                                         float64
Point de rosée                                                      float64
Humidité                                                            float64
Temps présent                                                       float64
Pression station                                                    float64
Rafales sur une période                                             float64
Periode de mesure de la rafale                                      float64
Précipitations dans la dernière heure                               float64
Précipitatio

In [102]:
# group rows by Date and applying the mean operation
data_meteo = data_meteo.groupby(['Date'], as_index=False).agg(np.mean)
#data = data.groupby(['Date']).agg(np.mean)
#data = data.loc['2019-12-01':]
print(data_meteo.shape)
data_meteo

(29003, 18)


,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Point de rosée,Humidité,Temps présent,Pression station,Rafales sur une période,Periode de mesure de la rafale,Précipitations dans la dernière heure,Précipitations dans les 3 dernières heures,Précipitations dans les 6 dernières heures,Précipitations dans les 12 dernières heures,Type de tendance barométrique.1,Température (°C)
0,2010-01-01 01:00:00+01:00,99103.333333,-147.5,7.750000,192.5,2.800,280.875,89.00,6.000000,96062.5,6.550,-10.0,0.200000,0.200000,0.250000,NaN,7.750000,9.475
1,2010-01-01 04:00:00+01:00,98823.333333,-282.5,7.000000,222.5,5.675,280.200,87.75,31.666667,95780.0,8.325,-10.0,0.200000,0.250000,NaN,NaN,7.000000,8.950
2,2010-01-01 07:00:00+01:00,98866.666667,-90.0,6.333333,165.0,7.825,278.525,85.75,21.000000,95772.5,11.300,-10.0,0.000000,0.800000,NaN,1.333333,6.333333,7.625
3,2010-01-01 10:00:00+01:00,99106.666667,232.5,2.000000,177.5,5.550,278.550,86.25,45.000000,96005.0,9.900,-10.0,0.350000,0.850000,NaN,NaN,2.000000,7.600
4,2010-01-01 13:00:00+01:00,99226.666667,107.5,1.000000,257.5,8.350,278.750,82.00,35.250000,96112.5,12.475,-10.0,0.100000,0.400000,1.250000,NaN,1.000000,8.525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28998,2020-02-02 10:00:00+01:00,102306.666667,122.5,1.750000,172.5,4.200,279.600,80.00,1.500000,99210.0,8.475,-10.0,0.000000,0.000000,0.000000,0.000000,1.750000,9.875
28999,2020-02-02 13:00:00+01:00,102210.000000,-80.0,8.000000,107.5,4.150,282.225,75.00,1.000000,99130.0,7.725,-10.0,-0.033333,-0.033333,-0.033333,-0.033333,8.000000,13.400
29000,2020-02-02 16:00:00+01:00,102090.000000,-105.0,5.750000,187.5,5.650,283.450,75.75,16.500000,99025.0,8.600,-10.0,-0.033333,-0.033333,-0.033333,-0.033333,5.750000,14.575
29001,2020-02-02 19:00:00+01:00,102183.333333,87.5,2.750000,177.5,5.900,282.875,81.00,0.500000,99112.5,10.600,-10.0,0.000000,-0.033333,-0.033333,-0.033333,2.750000,12.925


In [103]:
# adding custom dates columns
# to match the consumption dataset
data_meteo.insert(loc=1, column='Date - Heure', value=data_meteo['Date'])
data_meteo['Date'] = data_meteo['Date - Heure'].dt.date
data_meteo.insert(loc=1, column='Heure', value=data_meteo['Date - Heure'].dt.time)
data_meteo

KeyError: 'Date - Heure'

In [ ]:
# specific day values
#today = date.today().strftime("%Y-%m-%d")
target_date = datetime.strptime("2020-02-02", "%Y-%m-%d").date()
target_date_data = data_meteo.loc[data_meteo['Date'] == target_date]
target_date_data

In [ ]:
# average values for the given day
target_date_data.agg(np.mean)

In [ ]:
target_date_data[["Date - Heure", "Température (°C)", "Humidité"]].plot(x="Date - Heure", title="Average values for "+str(target_date), figsize=(10,5))

In [ ]:
# adding a year column to get average values per year
data_meteo.insert(loc=1, column='Annee', value=pd.to_datetime(data_meteo['Date'], format='%Y-%m-%d', utc=True).dt.year)
data_meteo.groupby(['Annee']).agg(np.mean)

In [ ]:
# representation part
# we differenciate the current year with incomplete data
current_year = datetime.now()
# temperature evolution since 2010
# gca = get current axis
ax = plt.gca()

data_meteo[['Date', 'Température (°C)']].plot(
    x='Date', label='Température', title="Evolution de la température (°C)", figsize=(15, 10), ax=ax
)

temperatures_stats = data_meteo[['Annee', 'Température (°C)']]
# TODO: deprecation : use agg().rename() instead
#temperatures_stats = temperatures_stats.groupby(['Annee'], as_index=False).agg({'Température (°C)':
#                                  {'Moyennes': np.mean, 'Minimales': np.min, 'Maximales': np.max}})

temperatures_stats = temperatures_stats.groupby(['Annee']).agg(Moyennes=('Température (°C)', np.mean),
                                                               Minimales=('Température (°C)', np.min),
                                                               Maximales=('Température (°C)', np.max))

temperatures_stats = temperatures_stats.reset_index()

temperatures_stats['Annee'] = pd.to_datetime(temperatures_stats['Annee'], format='%Y')

# display the plot
temperatures_stats[:-1].plot(
    x='Annee', label='Moyennes par an', ax=ax, marker='o'
)
# specific display for the current year (incomplete data)
temperatures_stats[-2:].plot(
    x='Annee', label='Moyennes par an', ax=ax, linestyle='dashed', marker='o'
)

In [ ]:
# wind speed during the last 10 years
data_meteo[['Date', 'Vitesse du vent moyen 10 mn']].plot(
    x='Date', label='Vitesse moyenne', title='Vitesse du vent moyen 10 mn', figsize=(10,5)
)

In [ ]:
# Humidity/rain statistics for the last 10 years
# TODO: improve readability
data_meteo[['Date', 'Type de tendance barométrique', 'Humidité', 'Précipitations dans les 12 dernières heures']].plot(
    x='Date', label='Vitesse moyenne', title='Average values related to rain', figsize=(10,5)
)